# Tensor Factorization

Have you ever wondered how Facebook does facial recognition? 
Or how to handle the data coming from thousands of IoT devices? 

In this lesson we'll cover the necessary mathematical techniques needed to work on these sorts of problems. To do so, we will need to generalize the concept of matrices to higher dimensions and resolve the implications this has on the tools, formulas, and theories we use in 2-D space. This requires building mathematical machinery to help us with these generalized matrices, called tensors, in order to be able to reconstruct them using a similar framework to singular value decomposition.

### Facial Recognition
<img src = 'faces.png'>

### Reconstructing data from sparse tensors in IoT 
<img src = 'IoT.png'>

## Review of Singular Value Decomposition

In [15]:
import logging
from scipy.io.matlab import loadmat
import sktensor #https://github.com/mnick/scikit-tensor/tree/master/sktensor
from sktensor import dtensor, cp_als, tucker
import numpy as np
from IPython.display import Latex, Image

### Statement of Theorem

Given a matrix $\mathbf{A} \in \mathbb{R}^{m\times n}$ , there exists a factorization of $\mathbf{A}$ of the form

$$\begin{align}
\mathbf{A} &= \mathbf{U\Sigma V^T}
\end{align}$$

where $\mathbf{U}$ is an $m\times m$ unitary matrix,
$\mathbf{\Sigma}$ is an $m\times n$ diagonal matrix, and
$\mathbf{V}$ is an $n\times n$ unitary matrix

<img src = 'SVD.png'>

Recall that this means we can split the matrix $A$ into a **sum** of its parts!

<img src ='SVDtrunc.png'>

Let's look at a basic example of this.

In [3]:
A = np.matrix([[1,0,0,0,2],[0,0,3,0,0],[0,0,0,0,0],[0,2,0,0,0]])

U,s,V = np.linalg.svd(A,full_matrices=False)

S = np.diag(s)

printred('A = \n')
print(A)
print('')

printred('SVD = \n')
print(np.dot(U,np.dot(S,V)))


A = 

[[1 0 0 0 2]
 [0 0 3 0 0]
 [0 0 0 0 0]
 [0 2 0 0 0]]

SVD = 

[[ 1.  0.  0.  0.  2.]
 [ 0.  0.  3.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.]]


## Tensor Notation and Definitions

**Definition:** A **Tensor** is a multidimensional array.

A 3-D tensor would look like this:

<img src = 'tensor.png'>

**Defintion:** The **mode-$k$ unfolding** of a tensor $A \in \mathbb{R}^{I_1 \times \cdots \times I_N}$ is a matrix $A_k \in \mathbb{R}^{I_k\times \Pi_{j\neq k} I_j}$ 

<img src ='unfolding.png'>

Here is an example that demonstrates the nuances of unfolding.

In [4]:
A = np.linspace(1,24, 24)
A = np.reshape(A, (2,3,4))

## Skills Check! What size should these modes be?

# Mode-1 matrix
A1 = np.reshape(A, (A.shape[0], A.shape[1]*A.shape[2]))
printred('mode-1 matrix A1 =\n')
print(A1)
print('')

# Mode-2 matrix
A2 = np.reshape(A, (A.shape[1], A.shape[0]*A.shape[2]))
printred('mode-2 matrix A2 =\n')
print(A2)
print('')

# Mode-3 matrix
A3 = np.reshape(A, (A.shape[2], A.shape[0]*A.shape[1]))
printred('mode-3 matrix A3 =\n')
print(A3)

mode-1 matrix A1 =

[[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.]
 [ 13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.]]

mode-2 matrix A2 =

[[  1.   2.   3.   4.   5.   6.   7.   8.]
 [  9.  10.  11.  12.  13.  14.  15.  16.]
 [ 17.  18.  19.  20.  21.  22.  23.  24.]]

mode-3 matrix A3 =

[[  1.   2.   3.   4.   5.   6.]
 [  7.   8.   9.  10.  11.  12.]
 [ 13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.]]


## Tensor Products

In order for us to work with tensors, we need to define certain operations on them, e.g., addition, multiplication. Fortunately, additional on tensors follows trivially from matrices, but there are some differences in how we define multiplication since we are in higher dimensional space. So how do we multiply tensors and matrices or tensors and vectors? 

**Definition:** The product of a tensor $X \in \mathbb{R}^{I_1 \times\cdots\times I_N}$ times a matrix $U \in \mathbb{R}^{J\times I_k}$ is denoted by $X\times_k U$ and is of size $I_1\times \cdots \times I_{k-1}\times J\times I_{k+1}\times\cdots\times I_N$

In [5]:
#EXAMPLE: Matrix multiplication of a (3,4,2) tensor with a (2,3) matrix along mode-1
# Declare matrix and tensor

## Skills Check! What should the size of the resulting tensor be?


X = np.linspace(1,24,24)
X = np.reshape(X,(3,4,2))
U = np.linspace(1,6,6)
U = np.reshape(U,(2,3))

#create tensor object
X = dtensor(X) 

# slice matrix by its frontal dimension
X_11 = X[:,:,0]
X_12 = X[:,:,1]

# Tensor multiplication via numpy
Y = np.zeros((2,4,2))
Y[:,:,0] = np.dot(U,X_11)
Y[:,:,1] = np.dot(U,X_12)
printred('Y =\n')
print(Y)
print(' ')

# using the sktensor package
Z = X.ttm(U,0)
printred('Z =\n')
print(Z)

# check shape
print('')
print('Shape of tensor is %s' % str(Z.shape))

Y =

[[[  70.   76.]
  [  82.   88.]
  [  94.  100.]
  [ 106.  112.]]

 [[ 151.  166.]
  [ 181.  196.]
  [ 211.  226.]
  [ 241.  256.]]]
 
Z =

[[[  70.   76.]
  [  82.   88.]
  [  94.  100.]
  [ 106.  112.]]

 [[ 151.  166.]
  [ 181.  196.]
  [ 211.  226.]
  [ 241.  256.]]]

Shape of tensor is (2, 4, 2)


**Definition:** The product of a tensor $X \in \mathbb{R}^{I_1 \times\cdots\times I_N}$ times a vector $\mathbf{v} \in \mathbb{R}^{I_k}$ is denoted by $X\bullet_k \mathbf{v}$ and is of size $I_1\times \cdots \times I_{k-1}\times I_{k+1}\times\cdots\times I_N$

In [6]:
#EXAMPLE: Vector multiplication of a (3,4,2) tensor with a vector of length 4
# Declare vector and tensor
X = np.linspace(1,24,24)
X = np.reshape(X,(3,4,2))
v = np.linspace(1,4, 4)

# Create tensor object
X = dtensor(X)

# slice matrix by its frontal dimension
X_11 = X[:,:,0]
X_12 = X[:,:,1]

# Tensor multiplication via numpy
Y = np.zeros((3,2))
Y[:,0] = np.dot(X_11,v)
Y[:,1] = np.dot(X_12,v)
printred('Y=\n')
print(Y)

# using sktensor package
Z = X.ttv(v,1)
printred('\nZ=\n')
print(Z)

Y=

[[  50.   60.]
 [ 130.  140.]
 [ 210.  220.]]

Z=

[[  50.   60.]
 [ 130.  140.]
 [ 210.  220.]]


### Tensor outer product

**Definition:** The **vector outer product** of two vectors $\mathbf{a}$ and $\mathbf{b}$ is denoted by $\mathbf{a}\odot\mathbf{b}$. Note that this result is a **matrix**. Hence, if $M = \mathbf{a}\odot\mathbf{b}$ then the entries of $M$ are
$$m_{ij} = a_i b_j$$

**Remark:** Outer products should not be a new concept. This outer product is common in 2-D space, e.g., consider an $(m\times 1)$ vector multiplied by a $(1\times n)$ vector.

## Rank of a tensor

The rank of a tensor causes difficulties when attempting to generalize matrix properties to higher order tensors. There are several possible generalizations of the notion of rank and we discuss some of them below.

**Definition:** An $N$-way tensor, $X\in \mathbb{R}^{I_1\times\cdot\times I_N}$, is **rank one** if it can be written as the outer product of $N$ vectors, i.e.,

$$X = \mathbf{a}^{(1)}\odot \mathbf{a}^{(2)} \odot\cdots\odot \mathbf{a}^{(N)}$$

An example of a rank one 3-way tensor looks like the figure below

<img src ='rankOneTensor.png'>

**Definition:** The **$n$-rank** of a tensor $X$, for $n = 1,\ldots,N$, denoted by $\operatorname{rank}_n(X)$, is the rank of the unfolding $X_n$.

$$\operatorname{rank}_n(X) := \operatorname{rank}(X_n)$$

**Definition:** The (outer-product) **rank of $X$**, denoted by $\operatorname{rank}(X)$, is defined as the smallest integer $r$ such that $$X = \sum_{i=1}^r a_1^i \otimes a_2^i \otimes \cdots \otimes a_N^i$$

**In other words**, the rank of $X$ is the minimum number of rank-1 tensors that can sum up to $X$

Visually, this looks like the figure below:
<img src = 'parafac.png'>

## Is there a higher order analogue  of SVD for tensors?

Ideally, we would like to have a higher order analogue of SVD, but for tensors. Their roles would be similar to that of SVD and would allow similar numerical approximations, lower-rank SVDs that reduce memory consumption, allow for principal component analysis, and have increased computational speed. Since the definition of rank is much more complex for tensors than for matrices, this leads us to some interesting results about tensor SVDs.


**Question:** Under what condition on the rank does a tensor allow a diagonal decomposition?

## Differences between SVD and HO-SVD

While the names might be similar, there are many differences between SVD for matrices and HO-SVD for tensors.

* The core tensor in HO-SVD is not necessarily diagonal like $\Sigma$ is in SVD; it may even be dense
* There is no finite algorithm for computing the rank of a tensor
* No algorithm for computing the upper bound of the rank of a tensor
    * 100% of $2\times 2$ matrices have rank 2
    * 79% of $2\times 2\times 2$ tensors have rank 2
    * 21% of $2\times 2\times 2$ tensors have rank 3

If the core tensor is diagonal, then the tensor SVD looks like this:
<img src = 'HOSVD.png'>

## Tucker Decomposition

If the core tensor in SVD is **not** diagonal, then we can calculate a different type of SVD, called a Tucker Decomposition:
<img src = 'tucker.png'>

The Tucker decomposition (HO-SVD) algorithm can be built from several SVDs as follows:

1. Given a tensor $A \in \mathbb{R}^{I_1\times \cdots \times I_N}$ construct the mode-$k$ unfolding matrix $A_k$ 

2. Compute the SVD of $A_k = U_k \Sigma_k V_k^T$ and store the side matrices $U_k$

3. The core tensor $S$ is then the projection of $A$ onto the tensor basis formed by factor matrices $\{U_k\}_{n=1}^N$, i.e., $S = A \times_{n=1}^N U_k^T$

### Properties of Tucker Tensor

* Core $\mathcal{G}$ may be dense
* $A,B,C$ are generally orthonormal
* Not unique

## Non-Uniqueness of Tucker Decomposition

In [7]:
# Declaration of tensor
T = np.zeros((3, 4, 2))
T[:, :, 0] = [[ 1,  4,  7, 10], [ 2,  5,  8, 11], [3,  6,  9, 12]]
T[:, :, 1] = [[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]]
T = dtensor(T)

In [8]:
def myhosvd(T):

    T1 = np.reshape(T,(np.shape(T)[0],np.shape(T)[1]*np.shape(T)[2]))
    U1,S1,V1 = np.linalg.svd(T1)

    T2 = np.reshape(T, (np.shape(T)[1], np.shape(T)[0]*np.shape(T)[2]))
    U2,S2,V2 = np.linalg.svd(T2)

    T3 = np.reshape(T, (np.shape(T)[2], np.shape(T)[0]*np.shape(T)[1]))
    U3,S3,V3 = np.linalg.svd(T3)
    
    S = T.ttm(U1.T, 0).ttm(U2.T, 1).ttm(U3.T, 2)
    
    return S, U1, U2, U3

In [9]:
Y = tucker.hooi(T, [3, 4, 2], init='nvecs')
S, U1, U2, U3 = myhosvd(T)

Let's look at the results

In [10]:
printred('Y = \n')
print(Y)
print('')
printred('my HOSVD =\n')
print(S, U1, U2, U3)

Y = 

(dtensor([[[  6.96306024e+01,  -1.13159238e-02],
        [ -1.81302157e-02,  -6.91903815e+00],
        [ -8.85917216e-15,  -5.36191225e-14],
        [  2.02550947e-14,  -6.70103577e-15]],

       [[ -7.01201417e-02,  -1.61083107e+00],
        [ -7.83965156e-01,  -7.00974052e-01],
        [ -6.31538362e-15,  -5.56604952e-15],
        [ -1.74939015e-15,  -7.04039172e-16]],

       [[ -6.31266418e-15,   5.74500373e-14],
        [  2.88718602e-14,   2.43932168e-14],
        [ -1.35683120e-15,   5.12482366e-16],
        [ -9.30383510e-16,  -1.68608668e-16]]]), [array([[ 0.54117833,  0.7351594 , -0.40824829],
       [ 0.57662442,  0.02894161,  0.81649658],
       [ 0.6120705 , -0.67727619, -0.40824829]]), array([[ 0.34444166,  0.76246964, -0.        , -0.54772256],
       [ 0.44038579,  0.32566908, -0.40824829,  0.73029674],
       [ 0.53632993, -0.11113147,  0.81649658,  0.18257419],
       [ 0.63227407, -0.54793202, -0.40824829, -0.36514837]]), array([[ 0.35334125,  0.9354945 ],
    

It is clear that the above two solutions are different, but if they can reconstruct the original tensor, $T$, then we have proven that Tucker Decomposition is not unique

In [11]:
printred('my HOSVD = \n')
print(S.ttm(U1,0).ttm(U2,1).ttm(U3,2))
print('')
printred('SKTENSOR = \n')
print(Y[0].ttm(Y[1][0], 0).ttm(Y[1][1], 1).ttm(Y[1][2], 2))

my HOSVD = 

[[[  1.  13.]
  [  4.  16.]
  [  7.  19.]
  [ 10.  22.]]

 [[  2.  14.]
  [  5.  17.]
  [  8.  20.]
  [ 11.  23.]]

 [[  3.  15.]
  [  6.  18.]
  [  9.  21.]
  [ 12.  24.]]]

SKTENSOR = 

[[[  1.  13.]
  [  4.  16.]
  [  7.  19.]
  [ 10.  22.]]

 [[  2.  14.]
  [  5.  17.]
  [  8.  20.]
  [ 11.  23.]]

 [[  3.  15.]
  [  6.  18.]
  [  9.  21.]
  [ 12.  24.]]]


## Memory Consumption

In [12]:
import sys
d = 30
A = np.zeros((d,d,d))
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        for k in range(A.shape[2]):
            A[i,j,k] = 1./np.sqrt((i+1)**2 + (j+1)**2 + (k+1)**2)
            
            
printred('Size of A = \n')
print(sys.getsizeof(A))
print('')

A = dtensor(A)
Z = tucker.hooi(A, [d,d,d], init='nvecs')
printred('Size of Z = \n')
print(sys.getsizeof(Z))

Size of A = 

216128

Size of Z = 

64


## Computational Efficiency

Below we check for the computational speed of the two algorithms and compare them for small tensors and large tensors.

In [13]:
%timeit tucker.hooi(A, [d,d,d], init='nvecs')
%timeit myhosvd(A)

10 loops, best of 3: 18.2 ms per loop
1 loop, best of 3: 183 ms per loop


In [14]:
%timeit tucker.hooi(T, [3,4,2], init='nvecs')
%timeit myhosvd(T)

100 loops, best of 3: 6.2 ms per loop
1000 loops, best of 3: 411 µs per loop


## Summary

In this lecture, we generalized the notion of matrices to higher dimensional spaces. This generalization increased the complexity of rank, SVD, matrix multiplication, and vector multiplication. Furthermore, we explored the areas in which the diagonal SVD exists and when it has to be approximated by a Tucker Decomposition. The computational demands can be decreased by more efficient algorithms and or lower-rank decompositions.

And now we know one very crucial aspect of how facebook is able to do facial recognition so quickly. By decomposing higher dimensional matrices (facial data from images) into approximate lower rank cores (using HO-SVD), computational demands can remain practical and their able to tell you your name from just your face.

Helper Functions

In [ ]:
#from IPython.core.display import HTML
#def css_styling():
#    styles = open("custom.css", "r").read()
#    return HTML(styles)
#css_styling()

In [1]:
def printred(string):
    print('\x1b[1;31m'+'%s'%string +'\x1b[0m')

## References

Diagrams came from: http://www.cs.cmu.edu/~christos/TALKS/SIGMOD-07-tutorial/tensor3.pdf

Material came from the following sources:

ftp://ftp.esat.kuleuven.be/pub/SISTA/ida/reports/97-75.pdf

http://www.sandia.gov/~tgkolda/pubs/pubfiles/SAND2007-6702.pdf

http://www.chemometrics.ru/materials/presentations/wsc6/T09.pdf

https://en.wikipedia.org/wiki/Tensor_rank_decomposition
